# Demonstrate Interaction of pandas Dataframes and omnikeeper

In [1]:

import omnikeeper_client as okc
from omnikeeper_client import simple_traits, TraitDefinition, TraitAttributeDefinition, ATTRIBUTETYPE_TEXT, ATTRIBUTETYPE_INTEGER
import os
import uuid

define some nameings

In [2]:
trait_name = "python_client_demo.test"
layer_name = "testlayer"

as usual create our client

In [3]:
okapiclient = okc.OkApiClient(
    backend_url=os.getenv('OMNIKEEPER_URL'),
    client_id=os.getenv('OMNIKEEPER_AUTH_CLIENTID'),
    username=os.getenv('OMNIKEEPER_AUTH_USERNAME'),
    password=os.getenv('OMNIKEEPER_AUTH_PASSWORD'),
)


ensure trait ``python_client_demo.test`` exists, define a function to interact with traits - this is obviosly somthing to be pushed to api/sdk ;-)

In [5]:
okc.upsert_trait(okapiclient, TraitDefinition(trait_name, [
        TraitAttributeDefinition("id", "test.id", ATTRIBUTETYPE_INTEGER),
        TraitAttributeDefinition("array", "test.array", ATTRIBUTETYPE_TEXT),
        TraitAttributeDefinition("some_key", "test.mapped_some_key", ATTRIBUTETYPE_TEXT),
      ]))

{'manage_upsertRecursiveTrait': {'id': 'python_client_demo.test'}}

TODO: example broken from here on down -> fix

create a test DataFrame

In [ ]:
import pandas as pd

df_init = pd.DataFrame.from_records([
    {"id": 1, "array": ["a", "b"], "some_key": "Value 1"},
    {"id": 3, "array": ["c", "d"], "some_key": "another Value"},
])

df_init

now write data to layer ``testlayer`` (ensure it exists from previuos samples)

In [ ]:
# insert that new data, this will also delete all old trait entities, if there are any
simple_traits.bulk_replace(client, trait_name=trait_name, input=df_init, id_attributes=["id"], id_relations=[], write_layer=layer_name, filter={})

load dataframe again

In [ ]:
# get that data back out from omnikeeper
# what we get back has an index that's the CIID of the corresponding CI

df_work = simple_traits.get_all(client, trait_name=trait_name, layers=[layer_name])
df_work

modify our dataframe

In [ ]:
# change things in the returned data
# change single value of existing data
df_work.iat[0, df_work.columns.get_loc("array")] = ["a", "b", "z"]
# add a new row, using our own created CIID
df_work = pd.concat([df_work, pd.DataFrame([{"id": 4, "array": ["x"], "some_key": "the new guy"}], index=[str(uuid.uuid4())])])
# drop a row
df_work.drop([df_work.index[1]], inplace = True)

df_work

write it back

In [ ]:
simple_traits.set_all(client, trait_name=trait_name, input=df_work, write_layer=layer_name)


now compare it

In [ ]:
df_final = simple_traits.get_all(client, trait_name=trait_name, layers=[layer_name])

print(f"Dataframes equal?: {df_final.equals(df_work)}") 

df_final
